## Load Data

In [2]:

from function import *
import pandas as pd
PATHS = {
    'abandonment': r"D:\xarray\merged_chunk_2\*.nc",
    'feature':     "D:/xarray/aligned2/Feature_all/*.nc",
    'csv':         "aligned_for_training.csv",
    'test_output': "positive_samples_test_500.csv",
    'output':      "positive_samples_full_with_features.csv"
}
YEARS = [2018, 2020]
# 2. 读取 PV 站点并切片（测试模式）
# 2. 读取 PV 站点并切片（测试模式）
pv_df = load_pv_sites(PATHS['csv'], years=YEARS)
# Convert lon and lat columns to float32
pv_df['lon'] = pv_df['lon'].astype('float32')
pv_df['lat'] = pv_df['lat'].astype('float32')
# Rename 'year' to 'time' and convert to datetime64
pv_df = pv_df.rename(columns={'year': 'time'})
pv_df['time'] = pd.to_datetime(pv_df['time'], format='%Y')
pv_df

,unique_id,p_area,capacity_m,country,year,lon,lat
0,19077,385.150738,0.000000,NZL,2020,-177.929167,-29.245833
1,34741,30959.725410,2.160149,TON,2020,-175.187500,-21.187500
2,34740,38427.397160,0.000000,TON,2020,-175.179167,-21.154167
3,34743,39.668561,0.000000,TON,2020,-175.162500,-21.229167
4,19075,3401.711988,0.000000,TON,2020,-173.970833,-18.645833
...,...,...,...,...,...,...,...
78685,4873,140.626685,0.000000,NZL,2020,174.804167,-41.195833
78686,4874,140.626685,0.000000,NZL,2020,175.029167,-40.862500
78687,35248,10.442622,0.000000,NZL,2020,175.095833,-40.920833
78688,35251,67.873305,0.000000,NZL,2020,176.120833,-37.712500


## Process

In [1]:
from align import align_df_with_ds
import pandas as pd
import xarray as xr

# 加载光伏站点
def load_pv_sites(csv2018: str, csv2020: str):
    """
    Load PV site CSVs and combine into a single DataFrame without duplicates.
    """
    df18 = pd.read_csv(csv2018)
    df18["year"] = 2018
    df20 = pd.read_csv(csv2020)
    df20["year"] = 2020
    df = pd.concat([df18, df20], ignore_index=True)
    df = df.drop_duplicates(subset=["lat", "lon", "year"])
    return df

pv_2018=r'D:\xarray\aligned2\pv\PV_2018_for_aligning.csv'
pv_2020=r'D:\xarray\aligned2\pv\PV_2020_for_aligning.csv'

df=load_pv_sites(pv_2018,pv_2020)

# 读取参考栅格
ds_all = xr.open_mfdataset(
    "D:/xarray/aligned2/Feature_all/*.nc",
    combine  = "by_coords",
    chunks   = {"time":1, "lat":500, "lon":500},
    parallel = True
)
# 重命名列以匹配align函数的要求
df = df.rename(columns={
    'lon': 'longitude',
    'lat': 'latitude'
})
ds_all = ds_all.rename({'lon': 'longitude', 'lat': 'latitude'})

# # 使用align_df_with_ds进行对齐
# aligned_df = align_df_with_ds(df, ds_all, trim=False, inplace=True)



gogogo


c:\Users\zpy10\Anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 500. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\zpy10\Anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 500. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\zpy10\Anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 500. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\zpy10\Anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stor

In [3]:
print(ds_all.rsds.time)

<xarray.DataArray 'time' (time: 3)> Size: 24B
array(['2015-01-01T00:00:00.000000000', '2018-01-01T00:00:00.000000000',
       '2020-01-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    spatial_ref  int32 4B 0
    band         int32 4B 1
  * time         (time) datetime64[ns] 24B 2015-01-01 2018-01-01 2020-01-01
Attributes:
    long_name:      time
    standard_name:  time
    axis:           T


In [4]:
aligned_df

,unique_id,p_area,capacity_m,country,year,longitude,latitude
0,1,13592.361840,1.201704,GRC,2018,21.612500,38.112500
1,2,3411.622302,0.309810,DEU,2018,6.379167,50.195833
2,3,20889.852880,1.752301,GBR,2018,-0.895833,52.037500
3,4,4154.511624,0.397576,GRC,2018,21.654167,38.112500
4,5,4794.474966,0.440007,CZE,2018,14.279167,50.520833
...,...,...,...,...,...,...,...
103928,35268,109.944469,0.000000,SWE,2020,14.720833,63.145833
103929,35269,41371.089830,2.757758,FIN,2020,22.954167,62.845833
103930,35270,12767.612290,1.024186,FIN,2020,23.079167,60.862500
103931,35271,18.517843,0.000000,RUS,2020,30.370833,60.012500


In [ ]:
aligned_df.to_csv('./data/aligned_for_training0519.csv', index=False)

### 注：这里的表格名称已经改为名称为embedding系列字符，存储于data目录下

In [11]:
# Find duplicate records based on longitude and latitude
duplicates = aligned_df[aligned_df.duplicated(subset=['longitude', 'latitude'], keep=False)]

# Sort the duplicates by longitude and latitude for better readability
sorted_duplicates = duplicates.sort_values(by=['longitude', 'latitude'])

print(f"Found {len(duplicates)} records with duplicate longitude and latitude coordinates")
print("\nSample of duplicate records:")
display(sorted_duplicates.head(20))

# Count how many duplicates exist for each unique coordinate pair
duplicate_counts = duplicates.groupby(['longitude', 'latitude']).size().reset_index(name='count')
duplicate_counts = duplicate_counts.sort_values('count', ascending=False)

print("\nCount of duplicates per coordinate pair:")
display(duplicate_counts.head(20))

# Sort dataframe to prioritize 2020 records
aligned_df_sorted = aligned_df.sort_values(by=['longitude', 'latitude', 'year'])

# For each group of duplicates, keep the 2020 record if it exists, otherwise keep the first one
def keep_2020_or_first(group):
    if 2020 in group['year'].values:
        return group[group['year'] == 2020].iloc[0:1]
    else:
        return group.iloc[0:1]

aligned_df_no_duplicates = aligned_df.groupby(['longitude', 'latitude']).apply(keep_2020_or_first).reset_index(drop=True)

print(f"\nRemoved {len(aligned_df) - len(aligned_df_no_duplicates)} duplicate records")
print(f"Original dataframe size: {len(aligned_df)}")
print(f"Deduplicated dataframe size: {len(aligned_df_no_duplicates)}")

# Save the deduplicated dataframe to CSV
aligned_df_no_duplicates.to_csv('aligner_results.csv', index=False)
print("\nSaved deduplicated aligned coordinates to 'aligned_coordinates.csv'")


Found 44477 records with duplicate longitude and latitude coordinates

Sample of duplicate records:


,unique_id,p_area,capacity_m,country,year,longitude,latitude
26500,26501,66982.132520,5.989546,WSM,2018,-171.987500,-13.829167
103404,34744,135455.397600,7.490917,WSM,2020,-171.987500,-13.829167
31801,31802,83156.516450,7.411814,USA,2018,-159.579167,21.904167
102789,34129,77512.320450,4.680453,USA,2020,-159.579167,21.904167
23987,23988,173816.258900,13.716502,USA,2018,-159.304167,22.129167
102778,34118,196395.095900,10.243810,USA,2020,-159.304167,22.129167
31896,31897,4210.569594,0.405401,USA,2018,-158.154167,21.412500
31913,31914,7829.428666,0.753830,USA,2018,-158.154167,21.412500
29282,29283,49946.832940,4.433596,USA,2018,-158.012500,21.470833
31836,31837,19115.766490,1.696836,USA,2018,-158.012500,21.470833



Count of duplicates per coordinate pair:


,longitude,latitude,count
14207,102.212500,38.312500,18
14205,102.204167,38.312500,14
18180,136.362500,34.620833,12
15807,115.070833,31.237500,11
14192,102.120833,36.604167,11
1292,-77.412500,36.120833,11
14217,102.237500,36.429167,11
13996,98.929167,39.954167,10
15241,113.520833,31.762500,10
14252,102.387500,38.462500,10



Removed 25237 duplicate records
Original dataframe size: 103927
Deduplicated dataframe size: 78690

Saved deduplicated aligned coordinates to 'aligned_coordinates.csv'


In [17]:
df_align = pd.read_csv('aligned_for_training.csv')
df_align.head()


,unique_id,p_area,capacity_m,country,year,longitude,latitude
0,19077,385.150738,0.000000,NZL,2020,-177.929167,-29.245833
1,34741,30959.725410,2.160149,TON,2020,-175.187500,-21.187500
2,34740,38427.397160,0.000000,TON,2020,-175.179167,-21.154167
3,34743,39.668561,0.000000,TON,2020,-175.162500,-21.229167
4,19075,3401.711988,0.000000,TON,2020,-173.970833,-18.645833
